In [12]:
import sys; sys.path.insert(0, '../')
from matplotlib import style
style.use('seaborn-poster')
import matplotlib.pyplot as plt, numpy as np, networkx as nx, \
json, os

from Utils import IO


fn = '1548025318.5751357'
with open(f'../Data/{fn}/settings.json', 'r') as f:
    setting = json.load(f)

fileNames = []
for (root, dirs, files) in os.walk(f'../Data/{fn}'):
    for file in files:
        if file.endswith('pickle') and 'settings' not in file and 'data' not in file:
            fileNames.append(os.path.join(root, file))
print(len(fileNames))
print(setting)

3002
{'repeats': 10000, 'deltas': 100, 'nSamples': 1000, 'step': 10000, 'burninSamples': 0, 'pulseSizes': [0.1, inf], 'updateMethod': 'single', 'nNodes': 12, 'nTrials': 20}


In [15]:
sFiles = sorted(fileNames, key = lambda x: x.split('/')[-1].split('_')[0])
print(setting)

simSettings = {}

newFiles = []

from tqdm import tqdm_notebook as tqdm
for file in tqdm(sFiles):
#     print(file)
    # extact relevant data from files
    # pulse as last 
    pulse = file.split('=')[-1].replace('_', ":").replace(' ', '').rstrip('.pickle')
    
    
    # data is ordered; control, pulses -> next temp repeat
    r = file.split('/')[-3]
#     print('>', r); assert 0
#     print(pulse); assert 0

    if 'control' in file:
        condition = (r, file.split('/')[-2])
        simSettings[condition] = simSettings.get(condition, -1) + 1
    try:
        float(r)
        if 'data' in file:
            assert 0
        # deal with trial counter
        trial = simSettings[condition]
        newFile = f'trial={trial}_r={r}_{pulse}.pickle'
        root = os.path.join(*file.split('/')[:3])
        os.makedirs(os.path.join(root, 'data'), exist_ok = 1)
        graph = IO.loadPickle(file).graph
        if newFile not in os.listdir(os.path.join(root, 'data')):
            newFiles.append(newFile)

#             store = dict(mi = sd.mi, px = sd.px, \
#                          cpx = sd.conditional, \
#                          snapshots = sd.snapshots)
#             graph = sd.graph 
            sd = IO.loadPickle(file)
            
            if 'control' not in file:
                for i in 'mi conditional snapshots'.split(' '):
                    setattr(sd, i, []) 

            
            IO.savePickle(os.path.join(root, 'data', newFile), sd)
#         IO.savePickle(f')
        # increase
#         simSettings[condition] += 1
    except Exception as e:
        print(e)
        continue
    if 'control' in file:
#         simSetting
        trial += 1
    

{'repeats': 10000, 'deltas': 100, 'nSamples': 1000, 'step': 10000, 'burninSamples': 0, 'pulseSizes': [0.1, inf], 'updateMethod': 'single', 'nNodes': 12, 'nTrials': 20}


<ipython-input-15-48964b798988>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm(sFiles):


could not convert string to float: 'Data'
could not convert string to float: 'Data'






















































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [34]:
IO.loadPickle('../data2019-11-25T14:21:45.881769.pickle')

{'aucs': {}, 'data': {}, 'rdata': {}, 'settings': {}, 'coeffs': {}}

In [24]:
%load_ext autoreload
%autoreload 2
from PlexSim.Models.FastIsing import Ising
# recreate modelSettings dict
print(simSettings)
modelSettings = dict(magSide = '',\
                    updateType = 'single',\
                    nudgeType = 'constant',\
                    )

mags = IO.loadPickle(f'../Data/{fn}/mags.pickle')

ratios = {}
print(mags.keys())
for i, j in zip(mags.get('magRange'), mags.get('temperatures')):
    ratios[i] = j
    
equilibrium = dict(ratios = ratios)

# print(newFiles[0], equilibrium)


newSettings = setting.copy()
newSettings['equilibrium'] = equilibrium
from PlexSim.Models.FastIsing import Ising
graph = sd.graph
graph.__version__ = 1

# for line in nx.generate_multiline_adjlist(graph, ','):
#     print(line)
print(graph.nodes['depr'])
model = Ising(graph = graph)
newSettings['model'] = model
newSettings['graph'] = graph
print(setting)

print(fn)
# for i, j in setting['model'].mapping.items():
#     print(i,j)


# print(setting, '>>\n', newSettings)
# print(root)
# IO.savePickle(os.path.join(root, 'setting.back'), setting)
IO.savePickle(os.path.join(root, 'settings.pickle'),\
              newSettings)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{('0.8', 'control'): 19, ('0.5', 'control'): 19, ('0.2', 'control'): 19}
dict_keys(['temps', 'temperatures', 'magRange', 'mag'])
{'H': 0.0773681410461827, 'nudges': 0, 'state': 1, 'nudge': 0.0}
Using legacy loader
{'repeats': 10000, 'deltas': 100, 'nSamples': 1000, 'step': 10000, 'burninSamples': 0, 'pulseSizes': [0.1, inf], 'updateMethod': 'single', 'nNodes': 12, 'nTrials': 20}
1548025318.5751357
